# Aufgabe 1 - Akku-Abenteuer: Tobi's Optimale Routenplanung

Den Code immer nachvollziehbar kommentieren! Bitte beachtet, dass das Notebook von Anfang bis Ende ohne Fehler durchlaufen muss und dass die requirements.txt Datei aktualisiert wird. 

## Teilaufgabe a): lageplan.png laden und verarbeiten

In [1]:
from PIL import Image
import numpy as np

image = Image.open("lageplan.png").resize((21, 21), Image.NEAREST)
img_array = np.array(image)
color_map = {
    (255, 255, 255, 0): "out",
    (0, 0, 0, 255): "wall",
    (200, 113, 55, 255): "hall",
    (0, 255, 0, 255): "prof",
    (255, 255, 0, 255): "lab",
    (0, 0, 255, 255): "tea"
}
# find al uniqe colors
# colors = np.unique(image.reshape(-1, image.shape[2]), axis=0)

# change rgb values to color names
mapper = lambda x: color_map.get(tuple(x))
plan = np.apply_along_axis(mapper, 2, img_array)

plan = np.swapaxes(plan, 0, 1)  # flip x and y axis


In [2]:
from PIL import ImageDraw


def visualize_path(path, title):
    # Create a transparent overlay
    old_image = Image.open("lageplan.png")
    overlay = Image.new("RGBA", old_image.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)

    # Draw the path on the overlay
    for x, y in path:
        draw.rectangle([(x*20+5, y*20+5), (x*20+15, y*20+15)], fill=(255, 255, 255, 200))

    # Combine the original image with the overlay
    result = Image.alpha_composite(old_image.convert("RGBA"), overlay)

    result.save("outputs/" + title + ".png")

    # Display the result
    result.show()

## Teilaufgabe b): Breitensuche

In [8]:
from collections import deque

start = (3, 17)
goal = (1, 3)

def bfs(start, goal):
    queue = deque()
    queue.append((start, []))

    while queue:
        node, path = queue.popleft()
        if node == goal:
            path.append(goal)
            return path
        else:
            possible_moves = [(node[0]+1, node[1]), (node[0]-1, node[1]), (node[0], node[1]+ 1), (node[0], node[1]- 1)]
            for move in possible_moves:
                if plan[move] != ('wal' or 'out'):
                    queue.append((move, path + [node]))
visualize_path(bfs(start, goal), "bfs")

## Teilaufgabe c): A*-Algorithmus

In [4]:
def heuristic(node, goal):
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

cost = {
    "hal": 2,
    "tea": 3,
    "pro": 4,
    "lab": 5,
    "wet": 20,
}

def a_star(start, goal):
    queue = [(start, [], 0)]
    visited = np.zeros_like(plan, dtype=int)

    while queue:
        node, path, path_cost = queue.pop(0)
        visited[node] = 1
        if node == goal:
            path.append(goal)
            return path

        possible_moves = [(node[0]+1, node[1]), (node[0]-1, node[1]), (node[0], node[1]+ 1), (node[0], node[1]- 1)]
        possible_moves = filter(lambda x: plan[x] != ('wal' or 'out') and visited[x] == 0, possible_moves)
        moves = [(move, path + [node], path_cost + cost.get(plan[move], float("inf"))) for move in possible_moves]
        queue.extend(moves)
        queue = sorted(queue, key=lambda x:heuristic(x[0], goal) + x[2])

    '''     while sorted_moves:
            best_move = sorted_moves.pop(0)
            if plan[best_move] != ('wal' or 'out') and visited[best_move] == 0:
                queue.append((best_move, path + [node]))
                break'''

path = a_star(start, goal)
print(path)


[(3, 17), (2, 17), (1, 17), (1, 16), (1, 15), (1, 14), (1, 13), (2, 13), (3, 13), (3, 12), (3, 11), (3, 10), (3, 9), (3, 8), (3, 7), (3, 6), (3, 5), (3, 4), (3, 3), (2, 3), (1, 3)]


## Teilaufgabe d): Greedy Best First Search-Algorithmus

In [5]:
def gbf(start, goal):
    queue = [(start, [])]
    visited = np.zeros_like(plan, dtype=int)

    while queue:
        node, path = queue.pop(0)
        visited[node] = 1
        if node == goal:
            path.append(goal)
            return path

        possible_moves = [(node[0]+1, node[1]), (node[0]-1, node[1]), (node[0], node[1]+ 1), (node[0], node[1]- 1)]
        possible_moves = filter(lambda x: plan[x] != ('wal' or 'out') and visited[x] == 0, possible_moves)

        queue.extend([(move, path + [node]) for move in possible_moves])
        queue = sorted(queue, key=lambda x:heuristic(x[0], goal))

path = gbf(start, goal)
print(path)


[(3, 17), (2, 17), (1, 17), (1, 16), (1, 15), (1, 14), (1, 13), (2, 13), (3, 13), (3, 12), (3, 11), (3, 10), (3, 9), (3, 8), (3, 7), (2, 7), (1, 7), (1, 6), (1, 5), (1, 4), (1, 3)]


## Teilaufgabe e): Dusseliger Doktorand

In [6]:
visualize_path(a_star(start, goal), "a_star")
plan[3, 3:14] = 'wet'
visualize_path(bfs(start, goal), "bfs")
visualize_path(a_star(start, goal), "a_star_wet")
visualize_path(gbf(start, goal), "gbf")